#  <공지사항>
1. 구글 드라이브에서 작업할 데이터(2차 데이터)를 확인 및 활용하여 작업을 부탁드립니다.(해당 폴더 내에서 작업)
2. 작업에 활용되는 패키지 또는 데이터는 반드시 구글 클라우드의 참가팀 폴더에 업로드하여 작업 부탁드립니다.
3. "런타임 -> 런타임 유형변경"이 GPU로 설정되어 있는지 확인합니다.
4. 최대 세션 유지시간은 12시간입니다.
5. 90분 이상 비활성화인 경우 세션이 종료됩니다.
6. 시간별로 "저장-> 드라이브 저장"을 통해 작업물을 백업하시기 바랍니다.
7. 오류 발생시 Q&A게시판 또는 카카오톡을 통해 문의 부탁드립니다.
8. 구글 계정의 비밀번호 및 정보 변경을 금지합니다.
9. 입력 파일 및 출력 파일 양식 추후 별도 제공(8월 둘째 주 예정)
<BR>★★★모델기획서 작성 및 모델 구현 참고 사항★★★<Br>
 ① 언어 : 파이썬(기타 언어 사용 불가)<br>② 인프라 : 구글 코렙 GPU버전<br>③ 사용가능 패키지 : PIP, 아나콘다로 제한(외부 라이브러리 사용 불가)<br>
 ※ 사전(Pre)학습 외부모델 binery 업로드하여 활용 가능하나, fine tuning은 해당 환경에서 진행되야 합니다.
 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#공용함수

In [ ]:
# 수정금지: 타임스탬프용 함수
from datetime import datetime
def printt(*args,**kwargs):
  now = datetime.now()
  now_str = "{:02}:{:02}:{:02}".format(now.hour,now.minute,now.second)
  print(now_str, *args,**kwargs)
  return int(now.hour)*60*60+int(now.minute)*60+int(now.second)

#연관 패키지 설치 (다른작업 금지)

In [ ]:
#TODO: 해당 블럭에 패키지 설치하세요.
!pip install transformers

     |████████████████████████████████| 2.6 MB 7.9 MB/s 
     |████████████████████████████████| 895 kB 56.3 MB/s 
     |████████████████████████████████| 636 kB 62.1 MB/s 
     |████████████████████████████████| 3.3 MB 53.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
# 필요한 모듈 로딩
import re
import torch
torch.cuda.empty_cache()
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, ElectraForSequenceClassification, AdamW
from tqdm.notebook import tqdm
import torch.optim as optim

import random
import torch.backends.cudnn as cudnn
import pandas as pd
import numpy as np
import os
seed= 2021 # 재현성을 위한 시드값 고정
random.seed(seed)
np.random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)  # type: ignore
torch.backends.cudnn.deterministic = True  # type: ignore
torch.backends.cudnn.benchmark = False  # type: ignore

# 파일로딩 (다른작업 금지)

In [ ]:
#TODO: 해당 블럭에 필요 파일 로딩 코드 넣으시오.
datas_1=pd.read_excel('/content/drive/MyDrive/2-37번.윤태양(윤태양)/2-37번.윤태양(윤태양)/공모전_제공_데이터(1차).xlsx')
datas_2=pd.read_excel('/content/drive/MyDrive/2-37번.윤태양(윤태양)/2-37번.윤태양(윤태양)/공모전 제공 데이터(2차).xlsx')
input_data=pd.read_excel('/content/drive/MyDrive/2-37번.윤태양(윤태양)/2-37번.윤태양(윤태양)/★공지사항★/input_data.xlsx') # 삼성카드에서 준비한 Test셋이 들어가도록 경로를 바꿔주시면 됩니다

# 모델 학습

In [ ]:
# 수정금지: 타임스탬프
_model_build_start_time = printt("Model building: Start")
_model_build_start_time

09:18:55 Model building: Start


33535

In [ ]:
#TODO: 블럭에 모델 학습 - 빌딩 코드를 넣으세요. (시간측정 구간)

# GPU 사용
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

datas_1 = datas_1.rename({'최종분류(우선순위 가장 높은것 선택)':'target'}, axis='columns') #1차제공데이터의 최종분류(우선순위 가장 높은 것 선택)을 'target'이라는 변수명으로 변경 
datas_2 = datas_2.rename({'최종분류(우선순위 가장 높은것 선택)':'target'}, axis='columns') #2차제공데이터의 최종분류(우선순위 가장 높은 것 선택)을 'target'이라는 변수명으로 변경
input_case=input_data.iloc[:,:-1] # result_data를 위한 데이터 틀 , 결과값만 하나의 열로 추가할 예정
input_data = input_data.rename({'TEXT':'발화'}, axis='columns') # 편의성을 위해 input_data의 TEXT도 학습데이터와 같은 컬럼명으로 변경 
datas=pd.concat([datas_1,datas_2]) # 1차제공데이터와 2차제공데이터 통합
datas=datas.replace({'target': '중립'}, {'target': '중립>중립'}) # Target을 감성과 대상 두 부분으로 나누어 학습하며 감서모델과 대상모델 모두를 활용하여 중립을 분류하기 위함

In [ ]:
#특수문자 및 불필요한 문자 제거 
datas['발화'] = [re.sub('[^A-Za-z0-9가-힣]', ' ', s) for s in datas['발화']]
input_data['발화'] = [re.sub('[^A-Za-z0-9가-힣]', ' ', s) for s in input_data['발화']]

In [ ]:
datas['target1']=datas['target'].str[:2] # 감성분류 target
datas['target2']=datas['target'].str[3:] # 대상분류 target

In [ ]:
# 불만 = 0   칭찬 = 1  중립 = 2
list1 = [(datas['target1']== '불만'), (datas['target1']=='칭찬'), (datas['target1']=='중립')]
choicelist1 = [0,1,2]
datas['target1']=np.select(list1, choicelist1)

In [ ]:
# 대상분류 Target 레이블 인코딩
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
datas['target2']=label_encoder.fit_transform(datas['target2'])

In [ ]:
# 데이터를 '발화' 'target1' 'target2' 변수만 추출하여 사용
datas = datas[['발화', 'target1', 'target2']]

In [ ]:
# 1차와 2차 데이터를 구분 -> 1차 데이터는 모두 학습데이터로 사용하기 위함: 1차 데이터를 모두 학습데이터로 사용하는 것이 validation에서 더 좋은 성능을 보였기 때문(경험적 근거)
datas_1=datas.iloc[:1000,:]
datas_2=datas.iloc[1000:,:]

In [ ]:
# 2차 데이터만 가지고 train, validation 데이터를 나눔
from sklearn.model_selection import train_test_split
data=datas_2.iloc[:, :-1]
target=datas_2.iloc[:, -1]

x_train, x_valid, y_train, y_valid = train_test_split(data, target, test_size=0.2, shuffle=True, stratify=target, random_state=36)

In [ ]:
x_train=pd.DataFrame(x_train, columns=['발화', 'target1'])
y_train=pd.DataFrame(y_train, columns=['target2'])
x_valid=pd.DataFrame(x_valid, columns=['발화', 'target1'])
y_valid=pd.DataFrame(y_valid, columns=['target2'])

In [ ]:
train_s = x_train.join(y_train, how='left')
train = pd.concat([train_s, datas_1])
valid = x_valid.join(y_valid, how='left')

In [ ]:
train1=train[['발화', 'target1']]
train2=train[['발화', 'target2']]
valid1=valid[['발화', 'target1']]
valid2=valid[['발화', 'target2']]
input_data=input_data[['발화', 'INT']]

In [ ]:
tokenizers=AutoTokenizer.from_pretrained("monologg/koelectra-small-v2-discriminator")

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/486 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255k [00:00<?, ?B/s]

In [ ]:
class SAMSUNGDataset(Dataset):
  
  def __init__(self, data):
    self.dataset = data
    # 중복제거
    self.dataset.drop_duplicates(subset=['발화'], keep='first', inplace=True)
    self.tokenizer = tokenizers

    print(self.dataset)
  
  def __len__(self):
    return len(self.dataset)
  
  def __getitem__(self, idx):
    row = self.dataset.iloc[idx, 0:2].values
    text = row[0]
    y = row[1]

    inputs = self.tokenizer(
        text, 
        return_tensors='pt',
        truncation=True,
        max_length=250,
        pad_to_max_length=True,
        add_special_tokens=True
        )
    
    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask, y

In [ ]:
class TESTDataset(Dataset):
  
  def __init__(self, data):
    self.dataset = data
    # 중복제거
    #self.dataset.drop_duplicates(subset=['발화'], inplace=True)
    self.tokenizer = tokenizers

    print(self.dataset)
  
  def __len__(self):
    return len(self.dataset)
  
  def __getitem__(self, idx):
    row = self.dataset.iloc[idx, 0:2].values
    text = row[0]
    y = row[1]

    inputs = self.tokenizer(
        text, 
        return_tensors='pt',
        truncation=True,
        max_length=250,
        pad_to_max_length=True,
        add_special_tokens=True
        )
    
    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask, y

In [ ]:
train_dataset1 = SAMSUNGDataset(train1) # 중복제거 및 토큰화된 데이터
valid_dataset1 = SAMSUNGDataset(valid1)
train_dataset2 = SAMSUNGDataset(train2)
valid_dataset2 = SAMSUNGDataset(valid2)
test_dataset = TESTDataset(input_data) # 중복제거하지않음 -> test 데이터는 중복데이터가 있더라도 모든 결과값을 뽑아내야되기 때문

                                                     발화  target1
8501                                      검사합니다  수고하세요         1
6897                                      늘 친절하셔서 감사합니다        1
5327                        상담직원들 너무 친절하고 신속해서 항상 감사드려요        1
702                                             전화가잘안되서        0
87                                         대기시간이 너무나길어요        0
...                                                 ...      ...
995   카드분실된지 1분만에 카드를 통해서 연락받아 바로 찾았어요 잃어버린지도 몰랐는데 감...        1
996              수시로 6개월무이자할부와 링크혜택이 다른카드와 구별되는 기능이 편리함        1
997                                무이자할부 자주 많이 주셔서감사합니다        1
998                                 무이자혜택만족긴할부만족잘쓰고있습니다        1
999                                  무이자 혜택이 많아 잘쓰고있어요         1

[8019 rows x 2 columns]
                                                     발화  target1
6125                                           친절하고 편하다        1
7309                                    항상 친절한 안내 감사합니다        1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying

In [ ]:
# 사전학습된 Koelectra 모델을 사용, 학습속도를 고려하여 small 모델을 사용
model1 = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v2-discriminator", num_labels = 3).to(device)
model2 = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v2-discriminator", num_labels = 16).to(device)

Downloading:   0%|          | 0.00/55.1M [00:00<?, ?B/s]

Some weights of the model checkpoint at monologg/koelectra-small-v2-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v2-discriminator and are newly initialized

In [ ]:
epochs_1 = 3 # 감성분류에 대한 epoch
epochs_2 = 7 # 대상분류에 대한 epoch
batch_size = 60 # 배치사이즈

In [ ]:
optimizer_1 = AdamW(model1.parameters(), lr=1e-4) # 감성분류 옵티마이저
optimizer_2 = AdamW(model2.parameters(), lr=1e-4) # 대상분류 옵티마이저
train_loader1 = DataLoader(train_dataset1, batch_size=batch_size, shuffle=True) 
valid_loader1 = DataLoader(valid_dataset1, batch_size=batch_size, shuffle=False)
train_loader2 = DataLoader(train_dataset2, batch_size=batch_size, shuffle=True)
valid_loader2 = DataLoader(valid_dataset2, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# 감성분류
losses = []
accuracies = []

for i in range(epochs_1):
  total_loss = 0.0
  correct = 0
  total = 0
  batches = 0
  
  model1.train()
  for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader1):
    optimizer_1.zero_grad()
    y_batch = y_batch.to(device)
    y_pred = model1(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
    loss = F.cross_entropy(y_pred, y_batch)
    loss.backward()
    optimizer_1.step()

    total_loss += loss.item()

    _, predicted = torch.max(y_pred, 1)
    correct += (predicted == y_batch).sum()
    total += len(y_batch)

      
    batches += 1
    if batches % 100 == 0:
      print("Batch Loss:", total_loss, "Accuracy:", correct.float() / total)
  
  losses.append(total_loss)
  accuracies.append(correct.float() / total)
  print("Train Loss:", total_loss, "Accuracy:", correct.float() / total)

model1.eval()

test_correct = 0
test_total = 0
for input_ids_batch, attention_masks_batch, y_batch in tqdm(valid_loader1):
  y_batch = y_batch.to(device)
  y_pred = model1(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
  _, predicted = torch.max(y_pred, 1)
  test_correct += (predicted == y_batch).sum()
  test_total += len(y_batch)

print("Accuracy:", test_correct.float() / test_total)


  0%|          | 0/134 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Batch Loss: 49.98590037971735 Accuracy: tensor(0.8975, device='cuda:0')
Train Loss: 55.70373912900686 Accuracy: tensor(0.9121, device='cuda:0')


  0%|          | 0/134 [00:00<?, ?it/s]

Batch Loss: 12.527035960927606 Accuracy: tensor(0.9635, device='cuda:0')
Train Loss: 15.75930985622108 Accuracy: tensor(0.9668, device='cuda:0')


  0%|          | 0/134 [00:00<?, ?it/s]

Batch Loss: 7.128971186466515 Accuracy: tensor(0.9768, device='cuda:0')
Train Loss: 9.025960235856473 Accuracy: tensor(0.9781, device='cuda:0')


  0%|          | 0/30 [00:00<?, ?it/s]

Accuracy: tensor(0.9776, device='cuda:0')


In [ ]:
# 대상분류
losses = []
accuracies = []

for i in range(epochs_2):
  total_loss = 0.0
  correct = 0
  total = 0
  batches = 0
  
  model2.train()
  for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader2):
    optimizer_2.zero_grad()
    y_batch = y_batch.to(device)
    y_pred = model2(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
    loss = F.cross_entropy(y_pred, y_batch)
    loss.backward()
    optimizer_2.step()

    total_loss += loss.item()

    _, predicted = torch.max(y_pred, 1)
    correct += (predicted == y_batch).sum()
    total += len(y_batch)
      
    batches += 1
    if batches % 100 == 0:
      print("Batch Loss:", total_loss, "Accuracy:", correct.float() / total)
  
  losses.append(total_loss)
  accuracies.append(correct.float() / total)
  print("Train Loss:", total_loss, "Accuracy:", correct.float() / total)

model2.eval()

test_correct = 0
test_total = 0
for input_ids_batch, attention_masks_batch, y_batch in tqdm(valid_loader2):
  y_batch = y_batch.to(device)
  y_pred = model2(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
  _, predicted = torch.max(y_pred, 1)
  test_correct += (predicted == y_batch).sum()
  test_total += len(y_batch)

print("Accuracy:", test_correct.float() / test_total)


  0%|          | 0/134 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Batch Loss: 194.3752248287201 Accuracy: tensor(0.4743, device='cuda:0')
Train Loss: 228.18060690164566 Accuracy: tensor(0.5521, device='cuda:0')


  0%|          | 0/134 [00:00<?, ?it/s]

Batch Loss: 62.09134528040886 Accuracy: tensor(0.8498, device='cuda:0')
Train Loss: 78.28681969642639 Accuracy: tensor(0.8542, device='cuda:0')


...

Batch Loss: 19.826439317315817 Accuracy: tensor(0.9463, device='cuda:0')
Train Loss: 25.681090485304594 Accuracy: tensor(0.9496, device='cuda:0')


  0%|          | 0/134 [00:00<?, ?it/s]

Batch Loss: 14.417487282305956 Accuracy: tensor(0.9617, device='cuda:0')
Train Loss: 18.666921991854906 Accuracy: tensor(0.9627, device='cuda:0')


  0%|          | 0/30 [00:00<?, ?it/s]

Accuracy: tensor(0.9295, device='cuda:0')


In [ ]:
# 수정금지: 타임스탬프
_model_build_end_time = printt("Model building: Start")
print(_model_build_end_time - _model_build_start_time)

09:29:55 Model building: Start
660


#모델 분류

In [ ]:
# 수정금지: 타임스탬프
_test_start_time = printt("TEST: Start")

09:29:55 TEST: Start


In [ ]:
#TODO: 해당 블럭에 테스트 수행을 위한 코드를 넣으세요. (시간측정 구간)
#분석 파일은 tsv 파일로 제공되며, 제공되는 학습데이터 파일과 동일한 레이아웃(단, 정답 분류 =공백)으로 제공됩니다.

# 감성부분 분류
model1.eval()

test_pred1 = []
for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
  y_batch = y_batch.to(device)
  y_pred = model1(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
  _, predicted = torch.max(y_pred, 1)
  test_pred1.append(predicted)

  0%|          | 0/17 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
# 대상부분 분류
model2.eval()

test_pred2 = []
for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
  y_batch = y_batch.to(device)
  y_pred = model2(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
  _, predicted = torch.max(y_pred, 1)
  test_pred2.append(predicted)

  0%|          | 0/17 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
# 수정금지: 타임스탬프
_test_end_time = printt("Model building: Start")
print(_test_end_time - _test_start_time)

09:30:01 Model building: Start
6


# 결과출력

In [ ]:
#TODO:해당 블럭에 테스트 결과를 파일로 저장하는 코드를 넣으세요. (시간측정 제외)
#저장 파일은tsv 파일로 제공되는 학습데이터 파일과 동일한 레이아웃(단, 정답 분류 = 테스트 결과 도출된 양식)으로 저장

In [ ]:
# 감성분류값
# 첫번째 range()의 수치와 두번째 range()의 수치 곱한 값이 데이터의 크기입니다. 즉 지금 코드의 구성은 10000*60=600000까지의 테스트 데이터를 뽑을 수 있도록 구성되어있습니다.
# 테스트 데이터의 수를 가늠할 수 없기에 추가적으로 더 많은 결과값을 뽑아야한다면 첫번째 range() 수치를 늘려주시면 감사합니다.   
output1 = []
for i in range(10000): # 테스트 데이터수에 비례하여 늘려주시면 감사합니다.
  for j in range(60):  # 배치사이즈 크기
    try:
      a=test_pred1[i][j].to("cpu").numpy()
      output1.append(a)
    except:
      pass

# 대상분류값
output2 = []
for i in range(10000): # 테스트 데이터수에 비례하여 늘려주시면 감사합니다
  for j in range(60):  # 배치사이즈 크기
    try:
      a=test_pred2[i][j].to("cpu").numpy()
      output2.append(a)
    except:
      pass


In [ ]:
output1=pd.DataFrame(output1, columns=['INT1'])
output2=pd.DataFrame(output2, columns=['INT2'])

In [ ]:
# 대상 레이블 값들 확인
label_encoder.inverse_transform([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])

array(['고객서비스>상담시스템', '고객서비스>상담원', '기타', '삼성카드', '삼성카드>고객정보관리',
       '삼성카드>금융상품', '삼성카드>라이프서비스', '삼성카드>리스렌탈상품', '삼성카드>상담/채널',
       '삼성카드>생활편의서비스', '삼성카드>심사/한도', '삼성카드>청구입금', '삼성카드>카드상품',
       '삼성카드>할부금융상품', '삼성카드>혜택', '중립'], dtype=object)

In [ ]:
# 텍스트 분류값으로 변경
output1=output1.replace({'INT1': 0}, {'INT1': '불만'})
output1=output1.replace({'INT1': 1}, {'INT1': '칭찬'})
output1=output1.replace({'INT1': 2}, {'INT1': '중립'})

In [ ]:
output2=output2.replace({'INT2': 0}, {'INT2': '고객서비스>상담시스템'})
output2=output2.replace({'INT2': 1}, {'INT2': '고객서비스>상담원'})
output2=output2.replace({'INT2': 2}, {'INT2': '기타'})
output2=output2.replace({'INT2': 3}, {'INT2': '삼성카드'})
output2=output2.replace({'INT2': 4}, {'INT2': '삼성카드>고객정보관리'})
output2=output2.replace({'INT2': 5}, {'INT2': '삼성카드>금융상품'})
output2=output2.replace({'INT2': 6}, {'INT2': '삼성카드>라이프서비스'})
output2=output2.replace({'INT2': 7}, {'INT2': '삼성카드>리스렌탈상품'})
output2=output2.replace({'INT2': 8}, {'INT2': '삼성카드>상담/채널'})
output2=output2.replace({'INT2': 9}, {'INT2': '삼성카드>생활편의서비스'})
output2=output2.replace({'INT2': 10}, {'INT2': '삼성카드>심사/한도'})
output2=output2.replace({'INT2': 11}, {'INT2': '삼성카드>청구입금'})
output2=output2.replace({'INT2': 12}, {'INT2': '삼성카드>카드상품'})
output2=output2.replace({'INT2': 13}, {'INT2': '삼성카드>할부금융상품'})
output2=output2.replace({'INT2': 14}, {'INT2': '삼성카드>혜택'})
output2=output2.replace({'INT2': 15}, {'INT2': '중립'})

In [ ]:
# 최종분류값으로 결과 도출
output=output1.join(output2, how='inner')
output['INT']=output['INT1']+'>'+output['INT2']
output['INT']=np.where(output['INT'].str.contains('중립'),'중립', output['INT']) # 감성분류와 대상분류 둘중 하나라도 중립으로 분류하면 중립으로 분류 

In [ ]:
# result_data를 위한 데이터 틀과 결과값을 조인하여 최종 result_data 생성
result_data=input_case.join(output[['INT']], how='inner')

In [ ]:
result_data

,KEY1,KEY2,TEXT,INT
0,1,6438,상담직원과 빠른연결했슴좋겠어요,불만>고객서비스>상담시스템
1,2,6850,안기다리고. 빨리상담되었음좋겠다,불만>고객서비스>상담시스템
2,3,7294,너무 오래기다리고 상담이 늦어져요,불만>고객서비스>상담시스템
3,4,9484,상담직원과 통화가 매우 힘들다,불만>고객서비스>상담시스템
4,5,10350,상담받으려면오래기다리는게불편함,불만>고객서비스>상담시스템
...,...,...,...,...
995,9958,103,카드분실된지 1분만에 카드를 통해서 연락받아 바로 찾았어요 잃어버린지도 몰랐는데 감...,칭찬>삼성카드>청구입금
996,9961,2,수시로 6개월무이자할부와 링크혜택이 다른카드와 구별되는 기능이 편리함,칭찬>삼성카드>혜택
997,9962,4,무이자할부 자주 많이 주셔서감사합니다,칭찬>삼성카드>혜택
998,9963,5,무이자혜택만족긴할부만족잘쓰고있습니다,칭찬>삼성카드>혜택


In [ ]:
# result_data 드라이브에 저장
result_data.to_excel('/content/drive/MyDrive/result_data.xlsx', index=False)